In [2]:
import torch
import argparse
import pandas as pd
from pathlib import Path
from locationencoder.pe import SphericalHarmonics

## in this sample, we use the spherical harmonics with level=3

level=3
# Initialize the Spherical Harmonics model
sh = SphericalHarmonics(legendre_polys=level)  # Adjust degree of Legendre polynomials if needed

# Variables to iterate over, including the ones you're interested in
variables = [
    'MAX',
    'WDSP',
    'MIN',
    'SLP',
    'MXSPD',
    'DEWP'
]

for variable in variables:
    # Read the CSV file for the current variable
    station_data = pd.read_csv(f"/data/zzn/insitu/data/unique_primary_station_ids_{variable}_tune.csv")
    
    # Extract latitude and longitude for each station
    latitudes = station_data['latitude'].values
    longitudes = station_data['longitude'].values
    
    # Create a list to hold embeddings for this variable
    embeddings_for_variable = []
    
    # Process each station's coordinates and compute embedding
    for lat, lon in zip(latitudes, longitudes):
        lonlat = torch.tensor([[lon, lat]], dtype=torch.float32)  # Longitude first, then Latitude
        embedded_lonlat = sh(lonlat)  # Get the embedding from Spherical Harmonics
        # Append the resulting embedding to the list
        embeddings_for_variable.append(embedded_lonlat.squeeze(0))  # Remove the extra dimension
    # Convert the list of embeddings into a 2D tensor for this variable
    embeddings_tensor = torch.stack(embeddings_for_variable)  # Shape: [num_stations, embedding_dim]
    
    # Save the embeddings for this variable to a file
    embeddings_file = Path('/data/zzn/insitu/data') / f'{variable}_embeddings_{level}.pt'
    torch.save(embeddings_tensor, embeddings_file)
    print(f"Embeddings for {variable} saved to {embeddings_file}")



Embeddings for MAX saved to /data/zzn/insitu/nips/MAX_embeddings_3.pt
Embeddings for WDSP saved to /data/zzn/insitu/nips/WDSP_embeddings_3.pt
Embeddings for MIN saved to /data/zzn/insitu/nips/MIN_embeddings_3.pt
Embeddings for SLP saved to /data/zzn/insitu/nips/SLP_embeddings_3.pt
Embeddings for MXSPD saved to /data/zzn/insitu/nips/MXSPD_embeddings_3.pt
Embeddings for DEWP saved to /data/zzn/insitu/nips/DEWP_embeddings_3.pt


In [1]:
## pip install the healpy
import healpy as hp
import numpy as np
import torch
import argparse
import pandas as pd
from pathlib import Path
from locationencoder.pe import SphericalHarmonics
## refinement_level denotes the number of healpy nodes
refinement_level = 3
## level denots the spherical harmoncis level, which should be same as the level of station nodes
level = 3
nside = 2 ** refinement_level
n_pixels = hp.nside2npix(nside)
theta, phi = hp.pix2ang(nside, np.arange(n_pixels))
latitudes = 90 - np.degrees(theta)
longitudes = np.degrees(phi) - 180

# latitudes = station_data['latitude'].values
# longitudes = station_data['longitude'].values
sh = SphericalHarmonics(legendre_polys=level)  # Adjust degree of Legendre polynomials if needed
# Create a list to hold embeddings for this variable
embeddings_for_variable = []

# Process each station's coordinates and compute embedding
for lat, lon in zip(latitudes, longitudes):
    lonlat = torch.tensor([[lon, lat]], dtype=torch.float32)  # Longitude first, then Latitude
    embedded_lonlat = sh(lonlat)  # Get the embedding from Spherical Harmonics
    # Append the resulting embedding to the list
    embeddings_for_variable.append(embedded_lonlat.squeeze(0))  # Remove the extra dimension
# Convert the list of embeddings into a 2D tensor for this variable
embeddings_tensor = torch.stack(embeddings_for_variable)  # Shape: [num_stations, embedding_dim]

# Save the embeddings for this variable to a file
embeddings_file = Path('/data/zzn/insitu/data') / f'{refinement_level}_healpix_embeddings_level_{level}.pt'
torch.save(embeddings_tensor, embeddings_file)
print(f"Embeddings for {refinement_level} saved to {embeddings_file}")

Embeddings for 3 saved to /data/zzn/insitu/nips/3_healpix_embeddings_level_3.pt
